# This was only a first idea

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import itertools
import json
from pathlib import Path
from datetime import timedelta, datetime

import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

from wikipedia_cleanup.data_processing import read_file, get_data
from wikipedia_cleanup.baseline import next_change

from sklearn.metrics import precision_recall_fscore_support

In [ ]:
mp_drive_dir = Path("/media/hpi_share/")#Path("//FS23/projekte$/MP2021/MPWS2021/MPWS2021FN1")
mp_plot_dir = mp_drive_dir / "plots"
input_path = Path("../../../data")
input_data = list(input_path.rglob("*.json"))
files = [x for x in input_data if x.is_file()]
len(files) # total 580

In [ ]:
num_edits = 0
change_tuples = []
for file in tqdm(files[:2]):
    with open(file, 'r', encoding='utf-8') as f:
        for jsonObj in f:
            single_edit = json.loads(jsonObj)
            #entries.append(single_edit)
            key = single_edit['key']
            pid = single_edit['pageID']
            user = single_edit['username'] if 'username' in single_edit.keys() else None
            title = single_edit['pageTitle']#['key']
            changes = single_edit['changes']
            timestamp = single_edit['validFrom']
            edit_type = single_edit['type']
            for change in changes:
                name = change['property']['name']
                current_value = change['currentValue'] if 'currentValue' in change.keys() else None
                previous_value = change['previousValue'] if 'previousValue' in change.keys() else None
                change_tuples.append((key, pid, title, user, name, previous_value, current_value, timestamp, edit_type))


In [ ]:
data = pd.DataFrame(change_tuples, columns=['key', 'pageID', 'title', 'user','name', 'previous_value', 'current_value', 'timestamp', 'edit_type'])
data['timestamp'] = pd.to_datetime(data['timestamp']).dt.tz_localize(None)
cd_data = data[data['edit_type'] != 'UPDATE'].copy()
#cd_data['timestamp'] = cd_data['timestamp'].dt.date

In [ ]:
x = cd_data.groupby(["key"])

In [ ]:
for key, group in x:
    deletes = group[group['edit_type'] == 'DELETE']
    creates = group[group['edit_type'] != 'DELETE']
    if any(deletes['title'] != 'Kazakhs') and len(deletes) > 0:
        t = deletes.merge(creates, on='key')
        possible_candidates = t[(t['previous_value_x'] == t['current_value_y']) &
          (t['name_x'] != t['name_y']) & 
          (t['name_x'] != t['name_y']) & 
          (t['timestamp_x'] < t['timestamp_y']) &
          (t['timestamp_x'].dt.date == t['timestamp_y'].dt.date) & 
          (t['previous_value_x'] != '')
         ]
        print(possible_candidates)
        print()
    #for delete in deletes.iterrows():

